In [ ]:
using DataFrames
using ExcelReaders
using Plots
using DataValues
using XLSX


In [ ]:
data_matrix_ind_col1 = readxlsheet("/home/AD/faidy/JuliaStatsProject/data/raw/real_data/Données_EDF_240611/Colmatage/IND-COL-1.xls", "IND-COL-1")

In [ ]:
data_matrix_ind_col2 = readxlsheet("/home/AD/faidy/JuliaStatsProject/data/raw/real_data/Données_EDF_240611/Colmatage/IND-COL-2.xls", "IND-COL-2")

In [ ]:
typeof(data_matrix_ind_col1)

In [ ]:
isNA(x) = typeof(x) == DataValue{Union{}}
valeurs = replace( x -> isNA(x) ? missing : x, data_matrix_ind_col1)

data_matrix_ind_col1 = replace( x -> isNA(x) ? missing : x, data_matrix_ind_col1)
data_matrix_ind_col2 = replace( x -> isNA(x) ? missing : x, data_matrix_ind_col2)

In [ ]:
function MatrixToDataFrame(mat)
    DF_mat = DataFrame(
        mat[2:end, 1:end],
        string.(mat[1, 1:end])
    )
    return DF_mat
end

In [ ]:
df1 = MatrixToDataFrame(data_matrix_ind_col1)
df2 = MatrixToDataFrame(data_matrix_ind_col2)

In [ ]:
typeof(df1)

In [ ]:
df1.VALEUR

In [ ]:
df1 = dropmissing(df1, :VALEUR)
df2 = dropmissing(df2, :VALEUR)

In [ ]:
unites = unique(df1.UNITE)
sizes = []
for unite in unites
    unite_data = df1[df1.UNITE .== unite,:]
    sous_unites = unique(unite_data.SOUS_UNITE)
    for sous_unite in sous_unites
        sous_unite_data = unite_data[unite_data.SOUS_UNITE .== sous_unite,:]
        circuits = unique(sous_unite_data.CIRCUIT)
        for circuit in circuits
            circuit_data = filter(row -> row.PE .== "E9" && row.BR .== "C", sous_unite_data[sous_unite_data.CIRCUIT .== circuit,:])
            p = plot(circuit_data[circuit_data.PE .== "E9",:].HEURES_MAT, circuit_data[circuit_data.PE .== "E9",:].VALEUR, seriestype=:scatter, xlabel="HEURES_MAT", ylabel="VALEUR", label="IND-COL-1", legend=:topright)
            println(circuit_data)

            # Chemin du répertoire où vous souhaitez enregistrer le fichier (relatif au répertoire de travail actuel)
            directory = "/home/AD/faidy/JuliaStatsProject/plots"

            # Combiner le chemin de répertoire et le nom du fichier
            filename = joinpath(directory, "IND-COL1 $unite $sous_unite $circuit.png")

            # Sauvegarder le graphique dans le fichier spécifié
            savefig(p, filename)
            append!(sizes, size(circuit_data.HEURES_MAT)[1])
        end
    end
end

display(sizes)


In [ ]:

# Créer l'histogramme
histogram(sizes, bins=10, title="Histogramme de Sizes", xlabel="Taille", ylabel="Fréquence", label="Fréquence")

# Chemin du répertoire où vous souhaitez enregistrer le fichier (relatif au répertoire de travail actuel)
# directory = "/home/AD/faidy/JuliaStatsProject/plots"

# # Combiner le chemin de répertoire et le nom du fichier
# filename = joinpath(directory, "histogramme_sizes.png")

# Sauvegarder l'histogramme dans le fichier spécifié
# savefig(filename)

# println("Histogramme enregistré dans le fichier $filename")


In [ ]:
unites = unique(df2.UNITE)
sizes2 = []
for unite in unites
    unite_data = df2[df2.UNITE .== unite,:]
    sous_unites = unique(unite_data.SOUS_UNITE)
    for sous_unite in sous_unites
        sous_unite_data = unite_data[unite_data.SOUS_UNITE .== sous_unite,:]
        circuits = unique(sous_unite_data.CIRCUIT)
        for circuit in circuits
            circuit_data = filter(row -> row.PE .== "E9" && row.BR .== "C", sous_unite_data[sous_unite_data.CIRCUIT .== circuit,:])
            p = plot(circuit_data[circuit_data.PE .== "E9",:].HEURES_MAT, circuit_data[circuit_data.PE .== "E9",:].VALEUR, seriestype=:scatter, xlabel="HEURES_MAT", ylabel="VALEUR", label="IND-COL-1", legend=:topright)
            println(circuit_data)

            # Chemin du répertoire où vous souhaitez enregistrer le fichier (relatif au répertoire de travail actuel)
            directory = "/home/AD/faidy/JuliaStatsProject/plots"

            # Combiner le chemin de répertoire et le nom du fichier
            filename = joinpath(directory, "IND-COL-2 $unite $sous_unite $circuit.png")

            # Sauvegarder le graphique dans le fichier spécifié
            savefig(p, filename)
            append!(sizes2, size(circuit_data.HEURES_MAT)[1])
        end
    end
end

display(sizes2)


In [ ]:
# Créer l'histogramme
histogram(sizes2, bins=14, title="Hist du nb de données de IND-COL-2 avec BR=C PE=E9 d'un circuit fixé", xlabel="Taille", ylabel="Fréquence", label="Fréquence", titlefontsize=10)


In [ ]:
length(sizes2)

In [ ]:
df_one_two = vcat(df1, df2, cols = :union )


In [ ]:
dropmissing(df_one_two, :SONDE)
missing_rows = df_one_two[ismissing.(df_one_two.SONDE), :]


In [ ]:
nettoyages = readxlsheet("/home/AD/faidy/JuliaStatsProject/data/raw/real_data/Données_EDF_240611/Nettoyages/NETTOYAGES.xls", "NETTOYAGES")
df_nettoyages = MatrixToDataFrame(nettoyages)
df_nettoyages = df_nettoyages[:, [:UNITE, :SOUS_UNITE, :CIRCUIT, :NUMERO, :HEURES_MAT]]
df_nettoyages = rename!(df_nettoyages, :HEURES_MAT => :HEURES_NETT)


In [276]:
unites = unique(df_one_two.UNITE)
sizes2 = []
for unite in unites
    unite_data = df_one_two[df_one_two.UNITE .== unite,:]
    sous_unites = unique(unite_data.SOUS_UNITE)
    for sous_unite in sous_unites
        sous_unite_data = unite_data[unite_data.SOUS_UNITE .== sous_unite,:]
        circuits = unique(sous_unite_data.CIRCUIT)
        for circuit in circuits
            circuit_data = filter(row -> row.PE .== "E9" && row.BR .== "C", sous_unite_data[sous_unite_data.CIRCUIT .== circuit,:])
            if !isempty(circuit_data)
                ind_col_1 = circuit_data[ismissing.(circuit_data.SONDE),:]
                ind_col_2 = circuit_data[.!ismissing.(circuit_data.SONDE),:]
            else
                circuit_data = filter(row -> row.PE .== "E8" && row.BR .== "C", sous_unite_data[sous_unite_data.CIRCUIT .== circuit,:])
                ind_col_1 = circuit_data[ismissing.(circuit_data.SONDE),:]
                ind_col_2 = circuit_data[.!ismissing.(circuit_data.SONDE),:]
            end

            p = plot(ind_col_1.HEURES_MAT, ind_col_1.VALEUR, xlabel="HEURES_MAT", ylabel="VALEUR", label="IND-COL-1", legend=:topright, markershape=:circle, markercolor=:blue, linestyle=:dash)
            plot!(p, ind_col_2.HEURES_MAT, ind_col_2.VALEUR, xlabel="HEURES_MAT", ylabel="VALEUR", label="IND-COL-2", legend=:topright,  markershape=:diamond, markercolor=:red, linestyle=:dash)
            # Chemin du répertoire où vous souhaitez enregistrer le fichier (relatif au répertoire de travail actuel)
            heures_nettoyages = filter(raw -> raw.UNITE .== unite && raw.SOUS_UNITE .== sous_unite && raw.CIRCUIT .== circuit, df_nettoyages)
            vline!(p, heures_nettoyages.HEURES_NETT, linestyle=:dot, color=:black, linewidth=1.5, label=false)

            directory = "/home/AD/faidy/JuliaStatsProject/plots"

            # Combiner le chemin de répertoire et le nom du fichier
            filename = joinpath(directory, "IND-COL-1&2 $unite $sous_unite $circuit.png")

            # Sauvegarder le graphique dans le fichier spécifié
            savefig(p, filename)
            append!(sizes2, size(circuit_data.HEURES_MAT)[1])
        end
    end
end

display(sizes2)


198-element Vector{Any}:
 13
  5
  7
  5
 13
  8
  6
  8
 10
  8
  ⋮
 13
  5
  5
 11
  5
  5
 12
  5
  5

In [ ]:
function IND_COL_3_and_EncMatToDataFrame(mat)
    # Extraire les noms des colonnes
    column_names = [mat[1,1],"$(mat[2,3])_ENC", "$(mat[2,4])_ENC", "$(mat[2,5])_ENC", "$(mat[2,6])_ENC", "$(mat[2,15])_IND_COL_3", "$(mat[2,16])_IND_COL_3", "$(mat[2,17])_IND_COL_3", "$(mat[2,18])_IND_COL_3"]

    # Créer un DataFrame à partir des données extraites
    df = DataFrame()

    # Ajouter les données au DataFrame
    df[!, column_names[1]] = mat[2:end,1]
    df[!, column_names[2]] = mat[2:end,3]
    df[!, column_names[3]] = mat[2:end,4]
    df[!, column_names[4]] = mat[2:end,5]
    df[!, column_names[5]] = mat[2:end,6]

    df[!, column_names[6]] = mat[2:end,15]
    df[!, column_names[7]] = mat[2:end,16]
    df[!, column_names[8]] = mat[2:end,17]
    df[!, column_names[9]] = mat[2:end,18]

    return df
end


In [ ]:
using XLSX

# Spécifiez le chemin complet de votre répertoire
directory = "/home/AD/faidy/JuliaStatsProject/data/raw/real_data/Données_EDF_240611/Encrassement"

# Utilisez readdir() pour obtenir les noms des fichiers et des sous-répertoires
files = readdir(directory)

# Filtrer les fichiers pour ne garder que ceux qui ont l'extension .xlsx
xlsx_files = filter(file -> endswith(file, ".xlsx"), files)
xlsx_files = [file for file in xlsx_files if file != "~\$PERFOS_U2S4_3,48-3,47-3,49.xlsx" && file != "~\$PERFOS_U19S2_2,45-2,43-2,44.xlsx"]
# Initialiser un dictionnaire vide pour stocker les DataFrames
dict_df_enc = Dict{String, DataFrame}()

# Lire les fichiers .xlsx et afficher les premières lignes
for file in xlsx_files
    try
        # Lire la première feuille du fichier .xlsx
        df = IND_COL_3_and_EncMatToDataFrame(XLSX.readdata("$directory/$file", "Feuil1", "A1:V144"))
        dict_df_enc[file] = dropmissing(df, :HEURES_MAT)

    catch e
        display("Erreur lors de la lecture du fichier $file: $e")
    end
end


In [ ]:
length(dict_df_enc)
size(files)

In [ ]:
for file in xlsx_files
    p = plot(dict_df_enc[file].HEURES_MAT, dict_df_enc[file].C1_ENC, xlabel="HEURES_MAT", ylabel="VALEUR", label="C1", legend=:topright, markershape=:circle, markercolor=:blue, linestyle=:dash)
    plot!(p, dict_df_enc[file].HEURES_MAT, dict_df_enc[file].C2_ENC, xlabel="HEURES_MAT", ylabel="VALEUR", label="C2", legend=:topright,  markershape=:diamond, markercolor=:red, linestyle=:dash)
    plot!(p, dict_df_enc[file].HEURES_MAT, dict_df_enc[file].C3_ENC, xlabel="HEURES_MAT", ylabel="VALEUR", label="C3", legend=:topright,  markershape=:diamond, markercolor=:red, linestyle=:dash)

    # Nombre de lignes verticales à ajouter
    num_lines = 30

    # Positions des lignes verticales régulièrement espacées
    vertical_lines_positions = range(minimum(dict_df_enc[file].HEURES_MAT), stop=maximum(dict_df_enc[file].HEURES_MAT), length=num_lines)

    # Ajouter les lignes verticales au plot
    for pos in vertical_lines_positions
        vline!(p, [pos], linestyle=:dot, color=:lightgray, linewidth=1.5, label=false)
    end
    # Chemin du répertoire où vous souhaitez enregistrer le fichier (relatif au répertoire de travail actuel)
    directory = "/home/AD/faidy/JuliaStatsProject/plots"

    # Combiner le chemin de répertoire et le nom du fichier
    filename = joinpath(directory, "ENC $file.png")

    # Sauvegarder le graphique dans le fichier spécifié
    savefig(p, filename)
end

In [ ]:
for file in xlsx_files
    p = plot(dict_df_enc[file].HEURES_MAT, dict_df_enc[file].C1_IND_COL_3, xlabel="HEURES_MAT", ylabel="VALEUR", label="C1", legend=:topright, markershape=:circle, markercolor=:blue, linestyle=:dash)
    plot!(p, dict_df_enc[file].HEURES_MAT, dict_df_enc[file].C2_IND_COL_3, xlabel="HEURES_MAT", ylabel="VALEUR", label="C2", legend=:topright,  markershape=:diamond, markercolor=:red, linestyle=:dash)
    plot!(p, dict_df_enc[file].HEURES_MAT, dict_df_enc[file].C3_IND_COL_3, xlabel="HEURES_MAT", ylabel="VALEUR", label="C3", legend=:topright,  markershape=:diamond, markercolor=:red, linestyle=:dash)

    # Nombre de lignes verticales à ajouter
    num_lines = 30

    # Positions des lignes verticales régulièrement espacées
    vertical_lines_positions = range(minimum(dict_df_enc[file].HEURES_MAT), stop=maximum(dict_df_enc[file].HEURES_MAT), length=num_lines)

    # Ajouter les lignes verticales au plot
    for pos in vertical_lines_positions
        vline!(p, [pos], linestyle=:dot, color=:lightgray, linewidth=1.5, label=false)
    end
    # Chemin du répertoire où vous souhaitez enregistrer le fichier (relatif au répertoire de travail actuel)
    directory = "/home/AD/faidy/JuliaStatsProject/plots"

    # Combiner le chemin de répertoire et le nom du fichier
    filename = joinpath(directory, "IND_COL_3 $file.png")

    # Sauvegarder le graphique dans le fichier spécifié
    savefig(p, filename)
end

In [ ]:
nettoyages = readxlsheet("/home/AD/faidy/JuliaStatsProject/data/raw/real_data/Données_EDF_240611/Nettoyages/NETTOYAGES.xls", "NETTOYAGES")

In [ ]:
df_nettoyages = MatrixToDataFrame(nettoyages)

In [ ]:
df_nettoyages = df_nettoyages[:, [:UNITE, :SOUS_UNITE, :CIRCUIT, :NUMERO, :HEURES_MAT]]
df_nettoyages = rename!(df_nettoyages, :HEURES_MAT => :HEURES_NETT)

In [ ]:
length(df_one_two.HEURES_MAT)

In [ ]:
df_one_two_with_nett = vcat(df_one_two[:, [:UNITE, :SOUS_UNITE, :CIRCUIT, :NUMERO, :HEURES_MAT]], df_nettoyages, cols = :union )


In [ ]:
# df_one_two_with_nett = leftjoin(df_one_two[:, [:UNITE, :SOUS_UNITE, :CIRCUIT, :NUMERO, :HEURES_MAT]], df_nettoyages, on = [:UNITE, :SOUS_UNITE, :CIRCUIT, :NUMERO] )
